1\. **Radioactive decay chain**

${\rm Tl}^{208}$ decays to ${\rm Pb}^{208}$ with a half-life $\tau$ of 3.052 minutes. Suppose to start with a sample of 1000 Thallium atoms and 0 of Lead atoms.

* Take steps in time of 1 second and at each time-step decide whether each Tl atom has decayed or not, accordingly to the probability $p(t)=1-2^{-t/\tau}$. Subtract the total number of Tl atoms that decayed at each step from the Tl sample and add them to the Lead one. Plot the evolution of the two sets as a function of time  
* Repeat the exercise by means of the inverse transform method: draw 1000 random numbers from the non-uniform probability distribution $p(t)=2^{-t/\tau}\frac{\ln 2}{\tau}$ to represent the times of decay of the 1000 Tl atoms. Make a plot showing the number of atoms that have not decayed as a function of time

In [ ]:
import numpy as np
import pandas as pd
import scipy as sp
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
tau=3.052*60 #seconds
Ti=1000
Pb=0
t=np.arange(0,100,1)

def dec_pdf(t,tau):
    return 1-2**(-t/tau)

def p(t, tau):
    return (2**(-t/tau) * math.log(2)/tau)

def p_icdf(x,tau):
    t = -tau*np.log2(1-x)
    return t

not_decayed=[]
decayed=[]
i=1
plt.figure()

while Ti>1:
        p=dec_pdf(i,tau)
        n_dec=np.around(Ti*p)
        Ti=Ti-n_dec
        Pb=Pb+n_dec
        not_decayed.append(Ti)
        decayed.append(Pb)
        i+=1
t=np.arange(1,i,1)         
plt.plot(t, decayed, color='r', label='Decayed Atoms')
plt.plot(t, not_decayed,color='b',label='Not Decayed Atoms')
plt.xlabel('time [s]')
plt.ylabel('number of atoms')
plt.legend()
plt.show()         
        
u = np.random.random(10000)
v = p_icdf(u, tau) # apply the inverse of the CDF
N=1000

t = np.arange(0,1000,1) 

plt.figure()
plt.hist(v, histtype='step', bins=100, density=False, linewidth=2)
plt.plot(t, N*(1-dec_pdf(t,tau)), linewidth=2, color='red')
plt.axis([0, 1000, 0 ,1000])
plt.xlabel('time [s]')
plt.ylabel('N*atoms')
plt.title('Histogram of not decayed atoms distribution')
plt.show()

2\. **Monte Carlo integration: hit/miss vs mean value method**

Consider the function: 

$$f(x) =\sin^2{\left( \frac{1}{1-x} \right)}$$

* Plot the function and compute the integral of $f(x)$ between 0 and 2 with the hit/miss method. Evaluate the error of your estimate (hint: repeat the integral $N$ times, and from the distribution of the integrals take the mean value and the standard deviation, the latter rescaled by the appropriate factor)
* Repeat the integral with the mean value method. Evaluate the error and compare it with the previous one.

In [ ]:
def f(x):
    return (np.sin(1/(1-x)))**2

N = 100000
x=np.linspace(0,2,N)
plt.figure(figsize=(10, 6), dpi=80)
plt.plot(x,f(x),"b-")

#repeat the process for N times
count=0
for i in range(N):
    x=2*np.random.random() #x in range 0-2
    y=np.random.random()   #y in range 0-1
    if y<f(x): 
        count+=1
Int=2*count/N

#calculate the error repeting the computation in order to find the mean and the std
I=[]
for k in range(10):
    count = 0
    for i in range(N):
        x=2*np.random.random()
        y=np.random.random()
        if y<f(x): 
            count += 1   
    I.append(2*count/N)

mean = np.mean(I)
std = np.std(I)

print('Hit/Miss Method :', Int)
print('Mean=', mean)
print('Standard Deviation:',std)


# Mean Value Method
x = np.zeros(N)
y = np.zeros(N)
for i in range(N):
    x[i] = 2*np.random.random()  
    y[i] = f(x[i])  
a=-1
b=1
I2 = ((b-a)*np.sum(y))/N

# Error
sigma = (b-a)*(np.sqrt(np.var(y))/np.sqrt(N))
print('Mean value method: ', I2)
print('Error:', sigma)

3\. **Monte Carlo integration in high dimension**

* Compute the area of a circle of unit radius, by integrating the function:

$$
f(x,y)=
\left\{
\begin{array}{ll}
      1 & x^2+y^2\le 1 \\
      0 & {\rm elsewhere}
\end{array} 
\right.
$$

* Generalize the result for a 10D sphere.

In [ ]:
# For 2D
N=100000
c=0
d=2
r2=1
r1=-1

def circle(x,y):
    if x**2+y**2<=1:
        return 1
    else:
        return 0
def circled(r,d):
    count=0
    for i in range(d):
        count+=r[i]**2
    if count<=1:
        return 1
    else:
        return 0
    
for i in range(N): 
    x=np.random.random()
    y=np.random.random() 
    
    c+=circle(x,y)
        
area = (((r2-r1)*d)/N)*c
print('Area in 2D:',area)        
r=np.zeros(10)     
d=10
c2=0
for i in range(N):
    c=0
    for j in range(d):
        r[j]=np.random.random()
    
    c2+=circled(r,d)

I = (((r2-r1)**d)/N)*c2

print('Area in 10D:',I)

4\. **Monte Carlo integration with importance sampling** 

Calculate the value of the integral:

$$
I=\int_0^1 \frac{x^{-1/2}}{e^x+1} dx
$$

using the importance sampling method with $w(x)=1/\sqrt{x}$. You should expect a result around 0.84.

In [ ]:
def f(x):
    return (x**(-0.5))/(np.exp(x)+1) # the function
def w(x):
    return 1/(np.sqrt(x))  #the function

a=0
b=1
it=100000

s=[]
for i in range(it):
    x=np.random.uniform(a,b)**2
    s.append(f(x)/w(x))

I = (2*np.sqrt(b)-2*np.sqrt(a))*(np.sum(s)/it)

print('Result:', I)